In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import f1_score

import matplotlib.pylab as plt

In [2]:
df = pd.read_csv("Train.csv")
y = df.target
x = df[["Policy Start Date", "Policy End Date", "Gender", "Age", "First Transaction Date", "No_Pol", "Car_Category", "Subject_Car_Colour", "Subject_Car_Make", "LGA_Name", "State", "ProductName"]]
x

,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName
0,2010-05-14,2011-05-13,Male,30,2010-05-14,1,Saloon,Black,TOYOTA,NaN,NaN,Car Classic
1,2010-11-29,2011-11-28,Female,79,2010-11-29,1,JEEP,Grey,TOYOTA,NaN,NaN,Car Classic
2,2010-03-21,2011-03-20,Male,43,2010-03-21,1,Saloon,Red,TOYOTA,NaN,NaN,Car Classic
3,2010-08-21,2011-08-20,Male,2,2010-08-21,1,NaN,NaN,NaN,NaN,NaN,CarSafe
4,2010-08-29,2010-12-31,Entity,20,2010-08-29,3,NaN,NaN,NaN,Lagos,Lagos,Muuve
...,...,...,...,...,...,...,...,...,...,...,...,...
12074,2010-05-25,2011-05-24,Female,30,2010-05-25,1,NaN,Black,Range Rover,Ibeju/Lekki,Ibeju-Lekki,Car Classic
12075,2010-10-03,2011-10-02,Female,59,2010-10-03,1,NaN,NaN,NaN,NaN,NaN,Car Classic
12076,2010-10-10,2011-10-08,Male,34,2010-10-10,1,NaN,NaN,NaN,NaN,NaN,CarSafe
12077,2010-02-27,2011-02-26,NaN,120,2010-02-27,2,NaN,White,TOYOTA,Victoria Island,Lagos,CVTP


In [3]:
len(x.Car_Category)

12079

In [4]:
x.Car_Category.isnull().sum(axis = 0)

3738

In [5]:
x.loc[:, "Policy Start Date"] = pd.to_datetime(x["Policy Start Date"], format='%Y-%m-%d')
x.loc[:, "Policy End Date"] = pd.to_datetime(x["Policy End Date"], format='%Y-%m-%d')

C:\Users\barto\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [6]:
p = pd.read_csv("VariableDefinitions.csv")
p

,ID,Unique ID for the customer
0,Policy Start Date,Date policy started
1,Policy End Date,Date policy ended
2,Gender,Gender of the customer
3,Age,Age of the customer
4,ProductName,Name of Insurance policy
5,First Transaction Date,First date payment was made
6,No_Pol,Number of policies the customer has
7,Car_Category,Type of car
8,Subject_Car_Colour,Car colour
9,Subject_Car_Make,Car make


In [7]:
date = pd.to_datetime(x["Policy Start Date"], format = "%Y-%m-%d")

In [8]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12079 entries, 0 to 12078
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Policy Start Date       12079 non-null  datetime64[ns]
 1   Policy End Date         12079 non-null  datetime64[ns]
 2   Gender                  11720 non-null  object        
 3   Age                     12079 non-null  int64         
 4   First Transaction Date  12079 non-null  object        
 5   No_Pol                  12079 non-null  int64         
 6   Car_Category            8341 non-null   object        
 7   Subject_Car_Colour      5117 non-null   object        
 8   Subject_Car_Make        9603 non-null   object        
 9   LGA_Name                5603 non-null   object        
 10  State                   5591 non-null   object        
 11  ProductName             12079 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(8)
memo

#### load datasets
#### percentage of missing values of each feature (plot)
#### percentage of missing values per column in test set (plot)

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, shuffle = True, random_state = 2020, stratify = y)

In [11]:
train_data = lgb.Dataset(x_train, 
                         label=y_train, 
                         categorical_feature=["Policy Start Date", "Policy End Date", "Gender", "First Transaction Date",
                                      "Car_Category", "Subject_Car_Colour", "Subject_Car_Make", "LGA_Name", "State", "ProductName"], 
                         free_raw_data=False)
test_data = lgb.Dataset(x_test, 
                        label=y_test, 
                        categorical_feature=["Policy Start Date", "Policy End Date", "Gender", "First Transaction Date",
                                      "Car_Category", "Subject_Car_Colour", "Subject_Car_Make", "LGA_Name", "State", "ProductName"], 
                        free_raw_data=False)

In [12]:
parameters = {'objective': 'binary',
              'metric': 'f1_score',
              'is_unbalance': 'true',
              'boosting': 'gbdt',
              'num_leaves': 63,
              'feature_fraction': 0.5,
              'bagging_fraction': 0.5,
              'bagging_freq': 20,
              'learning_rate': 0.01,
              'verbose': -1
             }

In [14]:
model_lgbm = lgb.train(parameters,
                            train_data,
                            valid_sets=test_data,
                            num_boost_round=5000,
                            early_stopping_rounds=50)

C:\Users\barto\AppData\Local\Programs\Python\Python39\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\barto\AppData\Local\Programs\Python\Python39\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: Policy Start Date, Policy End Date, Gender, First Transaction Date, Car_Category, Subject_Car_Colour, Subject_Car_Make, LGA_Name, State, ProductName